In [62]:
import pandas_datareader as web
import matplotlib.pyplot as plt
import pandas as pd
from dask.distributed import Client
from dask.distributed import as_completed
import pandas_market_calendars as mcal
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV, KFold, cross_val_score, RepeatedKFold, train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np
import statsmodels.api as sm

from dask.distributed import Client
from dask.distributed import as_completed
from IPython.display import clear_output
import time 
from statsmodels.tsa.stattools import adfuller

import pmdarima as pm


from dask.distributed import Client
from dask.distributed import as_completed

pd.set_option("max_rows", 10)

start = "1970-01-01"

In [ ]:
#sp500_ancient = web.DataReader("^GSPC", "yahoo",start=start)
sp500 = web.DataReader("^GSPC", "yahoo",start=start)
vix = web.DataReader("VIXCLS","fred",start=start)
T10Y3M = web.DataReader("T10Y3M","fred",start=start)
FEDFUNDS = web.DataReader("FEDFUNDS","fred",start=start)


In [ ]:
adfuller(data)

In [ ]:
pmdarima.arima.nsdiffs(sp500['Adj Close'].asfreq('Q',method='Pad'),4)

In [ ]:
pmdarima.arima.ndiffs(sp500['Adj Close'].asfreq('Q',method='Pad'))

In [ ]:
data_ = sp500[['Adj Close']].asfreq('Q',method='Pad')

k_=10
cv_outer = KFold(n_splits=k_, shuffle=False)

outer_results = []
outer_models = []

first_sets = []
for trainv_ix, test_ix in cv_outer.split(data_):
    # split data
    
    X_trainv, X_test = data_.iloc[trainv_ix], data_.iloc[test_ix, :]
    first_sets.append([X_trainv, X_test])
    #y_trainv, y_test = y.iloc[trainv_ix], y.iloc[test_ix]


In [ ]:
def inner_arima(test_sets):
    
    k_=10
    
    cv_inner = KFold(n_splits=k_, shuffle=False)
    
    X_trainv = test_sets[0]
    X_test = test_sets[1]
    
    inner_results = list()
    inner_models = list()

    #good juncture to fork locally
    for train_ix, valid_ix in cv_inner.split(X_trainv):
        # split data
        X_train, X_valid = X_trainv.iloc[train_ix, :], X_trainv.iloc[valid_ix, :]
        #y_train, y_valid = y_trainv.iloc[train_ix], y_trainv.iloc[valid_ix]
        
        model = pm.auto_arima(X_train, seasonal=True, m=4)
        forecasts = model.predict(len(X_valid))  # predict N steps into the future
        
        #plt.figure(figsize  = (8,8))
        #plt.title('Forecast result and Test data')
        # Visualize the forecasts (blue=train, green=forecasts)
        #x = np.arange(y.shape[0])
        #plt.plot(x[:150], train, c='blue')
        #plt.plot(x[150:], forecasts, c='green', label = 'ARIMA Forecast')
        #plt.plot(x[150:], test, c='red', label = 'Test Data')
        #plt.legend()
        #plt.show()
        
        inner_models.append(model)

        #yhat = model.predict(X_valid[t])
        yhat = forecasts
        # evaluate the model
        acc = np.mean(abs(X_valid.values.ravel()-forecasts))
        # store the result
        inner_results.append(acc)
        # report progress
        #print('>acc=%.3f, est=%.3f, cfg=%s' % (acc, result.best_score_, result.best_params_))
        # summarize the estimated performance of the model

    #good juncture to fork locally
    model_scores = list()

    #test inner model as ansemble
    for m in inner_models:
        model_scores.append(np.mean(abs(m.predict(len(X_test))-X_test.values.ravel())))

    #average internal model results and report as outer_result which*
    return([np.mean(np.abs(model_scores)),inner_models])
    #outer_results.append(np.mean(np.abs(model_scores)))
    #outer_models.append(inner_models)

    #print('I Accuracy: %.3f (%.3f)' % (mean(inner_results), std(inner_results)))


In [ ]:

#*will in turn be averaged over all folds to determine that combination of term's outer score (real world performance, which will be our model evaluating metric).
score = [np.mean(outer_results), np.std(outer_results)]

#print('O Accuracy: %.3f (%.3f)' % (score[0], score[1]))
print(score)
print(outer_models)

    
    #print(X_trainv)
    #print(X_test)

In [ ]:
p_values1 = [0, 1, 2, 4, 6, 8, 10]
d_values1 = range(0, 3)
q_values1 = range(0, 3)

In [ ]:
print(sp500.index[0])
print(sp500.index[-1])
print("years:",len(sp500)/252)

fig = plt.figure() 

n = 8
# Figure size

fig.set_size_inches(30, 6)

#for d in [1,2,3,4,5,7,10,15,20,30,40,50,60,70,80]:
for d in [1]:
    clear_output(wait=False)
    diffl = d
    
    data = sp500['Adj Close'].asfreq('Q',method='Pad').pct_change(diffl).dropna()
    mean = np.mean(data)
    stddev = np.std(data)
    predictionstddev =  stddev*(1+(1/len(data)))    
    
    print("rolling return over x years:",diffl/4)
    print("windows:", len(data)/4)
    
    print("mean:",mean)
    #print("annual mean return:",np.power(1+mean,n))
    print("annual mean return:",np.sqrt(1+mean))
    print("standard dev:",stddev)

    result = adfuller(data)
    print('ADF Statistic: %f' % result[0])
    print('p-value: %f' % result[1])
    
    #print('Critical Values:')
    #for key, value in result[4].items():
        #print('\t%s: %.3f' % (key, value))    
        
    fig0, fig0_ax1 = plt.subplots()
    
    fig0.set_size_inches(30, 6)

    color = 'tab:red'
    fig0_ax1.set_xlabel('time (s)')
    fig0_ax1.set_ylabel('exp', color=color)
    #fig0_ax1.plot(data1, color=color)
    fig0_ax1.plot(data)
    movMean = data.rolling(window=n).mean()
    movStd = data.rolling(window=n*n,center=True,min_periods=1).std()
    #page 597 Statistics for Business and Economics
    predictionMovStd = movStd*(1+(1/n*n))
    #print(predictionMovStd)
    
    fig0_ax1.plot(movMean)
    fig0_ax1.plot(movMean+(predictionMovStd*2))
    #fig0_ax1.plot(movMean+(movStd*2))
    fig0_ax1.plot(movMean-(predictionMovStd*2))
    #fig0_ax1.plot(movMean-(movStd*2))
    fig0_ax1.axhline(y=mean, color='b', linestyle='-')
    #fig0_ax1.axhline(y=mean+(predictionstddev*2), color='b', linestyle='-')
    #fig0_ax1.axhline(y=mean-(predictionstddev*2), color='b', linestyle='-')
        
    plt.xticks(rotation = 60,size=16)
    fig0_ax1.tick_params(axis='y', labelcolor=color)

    fig0_ax2 = fig0_ax1.twinx()  # instantiate a second axes that shares the same x-axis

    color = 'tab:blue'
    fig0_ax2.set_ylabel('sin', color=color)  # we already handled the x-label with ax1
    fig0_ax2.plot(sp500['Adj Close'].asfreq('Q',method='Pad'),color='m')
    fig0_ax2.tick_params(axis='y', labelcolor=color)

    fig0.tight_layout()  # otherwise the right y-label is slightly clipped
    
    temp = pd.DataFrame(data.quantile(q=[0.02, .09, .25, .5, .75, .91, .98], interpolation='linear'))
    
    plt.axhline(y=temp.loc[.5][0], color='c', linestyle='-')
    plt.axhline(y=temp.loc[.02][0], color='y', linestyle='-')
    plt.axhline(y=temp.loc[.98][0], color='y', linestyle='-')
    
    plt.axhline(y=mean+(predictionstddev*2), color='r', linestyle='-')
    plt.axhline(y=mean-(predictionstddev*2), color='r', linestyle='-')
    
    plt.show()
        
    #display(temp)
    #display(pd.concat([temp,np.power(1+temp,n)],axis=1))
    display(pd.concat([temp,np.sqrt(1+temp)],axis=1))

    counts, bins = np.histogram(data)
    
    fig, ax1 = plt.subplots()
    color = 'tab:red'
    ax1.set_xlabel('time (s)')
    ax1.set_ylabel('exp', color=color)
    ax1.hist(bins[:-1], bins, weights=counts,histtype='step',color=color)
    #.plot(data1, color=color)
    plt.xticks(rotation = 60,size=16)
    ax1.tick_params(axis='y', labelcolor=color)
    
    #plt.show()
    
    ax2 = ax1.twinx()
    color = 'tab:blue'
    
    ax2.set_ylabel('sin', color=color)  # we already handled the x-label with ax1
    #ax2.scatter(y=temp.reset_index()['index'],x=temp.reset_index()['Adj Close'],alpha=.5,cmap='viridis',color='black')
    ax2.tick_params(axis='y', labelcolor=color)

    fig.tight_layout()  # otherwise the right y-label is slightly clipped
    #plt.show()

    temp.index = [0.02, .09, .25, .5, .25, .09, .02]
    ax2.scatter(y=temp.reset_index()['index'],x=temp.reset_index()['Adj Close'],alpha=.5,cmap='viridis',color='black')
    
    plt.show()
    
    #plt.cla()

In [ ]:
tickers = ['^FTSE', '^N225', '^GDAXI', '^FCHI', '^HSI', '^BVSP',
                   '^RUT', 'IXY', 'IXR', 'IXE', 'IXM', 'IXV', 'IXT',
                   'IXB', 'IXU', '^IRX', '^FVX', '^TNX', '^TYX', 'VWEHX',
                   'VFSTX', 'VWESX']

"""
def process_data(ticker):
    df = web.DataReader(ticker, "yahoo")
    return(df)

client = Client('192.168.3.100:8786',timeout=3)

future_ = client.map(process_data,tickers)
results = list()

for f in as_completed(future_):
        if(f.status=="error"):
            results.append("error")
        else:
            results.append(f.result())     

client.close()
"""

In [ ]:
#!/usr/bin/python
# -*- coding: utf-8 -*-

# ------------------------------------
# file: zca.py
# date: Thu May 21 15:47 2015
# author:
# Maarten Versteegh
# github.com/mwv
# maartenversteegh AT gmail DOT com
#
# Licensed under GPLv3
# ------------------------------------
"""zca: ZCA whitening with a sklearn-like interface

"""

from __future__ import division

import numpy as np
from scipy import linalg

from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.utils.validation import check_is_fitted
from sklearn.utils import check_array, as_float_array

class ZCA(BaseEstimator, TransformerMixin):
    def __init__(self, regularization=1e-6, copy=False):
        self.regularization = regularization
        self.copy = copy

    def fit(self, X, y=None):
        """Compute the mean, whitening and dewhitening matrices.

        Parameters
        ----------
        X : array-like with shape [n_samples, n_features]
            The data used to compute the mean, whitening and dewhitening
            matrices.
        """
        X = check_array(X, accept_sparse=None, copy=self.copy,
                        ensure_2d=True)
        X = as_float_array(X, copy=self.copy)
        self.mean_ = X.mean(axis=0)
        X_ = X - self.mean_
        cov = np.dot(X_.T, X_) / (X_.shape[0]-1)
        U, S, _ = linalg.svd(cov)
        s = np.sqrt(S.clip(self.regularization))
        s_inv = np.diag(1./s)
        s = np.diag(s)
        self.whiten_ = np.dot(np.dot(U, s_inv), U.T)
        self.dewhiten_ = np.dot(np.dot(U, s), U.T)
        return self

    def transform(self, X, y=None, copy=None):
        """Perform ZCA whitening

        Parameters
        ----------
        X : array-like with shape [n_samples, n_features]
            The data to whiten along the features axis.
        """
        check_is_fitted(self, 'mean_')
        X = as_float_array(X, copy=self.copy)
        return np.dot(X - self.mean_, self.whiten_.T)

    def inverse_transform(self, X, copy=None):
        """Undo the ZCA transform and rotate back to the original
        representation

        Parameters
        ----------
        X : array-like with shape [n_samples, n_features]
            The data to rotate back.
        """
        check_is_fitted(self, 'mean_')
        X = as_float_array(X, copy=self.copy)
        return np.dot(X, self.dewhiten_) + self.mean_


In [ ]:
nyse = mcal.get_calendar('NYSE')
early = nyse.schedule(start_date=sp500.index[0].strftime('%Y-%m-%d'), end_date=sp500.index[-1].strftime('%Y-%m-%d'))

In [ ]:
#ax2=ax.twinx()

# create figure and axis objects with subplots()
fig,ax2=plt.subplots()

ax2.plot(T10Y3M.index,T10Y3M.replace([np.inf, -np.inf], 0))
#plt.set_xlabel("T10Y3M")
ax2.set_ylabel("T10Y3M")
#ax2.set_ylabel("lifeExp")


ax2.plot(T10Y3M.index,T10Y3M.pct_change().replace([np.inf, -np.inf]))
#ax.set_xlabel("10Y3M-Delta")
plt.show()


In [ ]:
newData = pd.concat([vix.reindex(early.index),T10Y3M.reindex(early.index),sp500.reindex(early.index)['Adj Close'],FEDFUNDS.reindex(early.index)],axis=1).interpolate(method='time')

In [ ]:
newData.corr()

In [ ]:
newDataDelta = newData.pct_change().replace([np.inf, -np.inf], 0).dropna()
#newDataDelta.corr()
#newDataDelta = newData

In [ ]:
# create figure and axis objects with subplots()
fig,ax = plt.subplots()
# make a plot

# make a plot with different y-axis using second axis object
#ax2.plot(sp500.index, sp500['Adj Close'].diff(), marker=".",color="yellow")
ax.plot(newDataDelta.index, newDataDelta['Adj Close'], marker=".",color="blue")
ax.set_ylabel("SP500",color="blue",fontsize=8)

ax3=ax.twinx()
# make a plot with different y-axis using second axis object
#ax2.plot(sp500.index, sp500['Adj Close'].diff(), marker=".",color="yellow")
ax3.plot(newDataDelta.index, newDataDelta['T10Y3M'], marker=".",color="green")
ax3.set_ylabel("T10Y3M",color="green",fontsize=8)
plt.show()

fig,ax = plt.subplots()
# make a plot

# make a plot with different y-axis using second axis object
#ax2.plot(sp500.index, sp500['Adj Close'].diff(), marker=".",color="yellow")
ax.plot(newDataDelta.index, newDataDelta['Adj Close'], marker=".",color="blue")
ax.set_ylabel("SP500",color="blue",fontsize=8)

ax2=ax.twinx()

ax2.plot(newData.index, newData['VIXCLS'], marker=".", color="red")
# set x-axis label
#ax.set_xlabel("year",fontsize=14)
# set y-axis label
ax2.set_ylabel("VIX",color="red",fontsize=8)

plt.show()



#plt.scatter(newDataDelta['Adj Close'].pct_change(),newDataDelta['T10Y3M'].pct_change())
#plt.show()

In [ ]:
newDataDelta.columns

In [ ]:


corrscores = []

scores = []

def compare_x_lag_y(npa_):
                
    k_=10

    cv_outer = KFold(n_splits=k_, shuffle=False)
    cv_inner = KFold(n_splits=k_, shuffle=False)

    i = npa_[0]
    sp500_ = npa_[1]
    vix_ = npa_[2]
    formula = npa_[3]
    
    #default
    if(formula==0):
        combined = pd.concat([vix_.reindex(early.index),sp500_.reindex(early.index)['Adj Close'].shift(i)],axis=1).interpolate(method='time').dropna()
    #sp500 differenced twice
    if(formula==1):
        combined = pd.concat([vix_.reindex(early.index),sp500_.reindex(early.index)['Adj Close'].pct_change().shift(i)],axis=1).interpolate(method='time').dropna()
        
    #corrscores.append(combined.corr())
    
    X=combined[['VIXCLS']]
    y=combined[['Adj Close']]

    outer_results = list()

    for trainv_ix, test_ix in cv_outer.split(X):
        # split data
        X_trainv, X_test = X.iloc[trainv_ix], X.iloc[test_ix, :]
        y_trainv, y_test = y.iloc[trainv_ix], y.iloc[test_ix]

        inner_results = list()
        inner_models = list()

        #good juncture to fork locally
        for train_ix, valid_ix in cv_inner.split(X_trainv):
            # split data
            X_train, X_valid = X_trainv.iloc[train_ix, :], X_trainv.iloc[valid_ix, :]
            y_train, y_valid = y_trainv.iloc[train_ix], y_trainv.iloc[valid_ix]
            model = LinearRegression()
            #just getting hyper-parm's (fit)
            lm = model.fit(X_train,y_train)
            inner_models.append(lm)

            #print(pd.concat([X_train,y_train],axis=1).corr())
            #print(lm.coef_)
            #yhat = gam.predict(X_valid[t])
            # evaluate the model
            #acc = np.mean(abs(y_valid.values.ravel()-yhat))
            # store the result
            #inner_results.append(acc)

        #good juncture to fork locally
        model_scores = list()

        #test inner model as ansemble
        for m in inner_models:
            model_scores.append(np.mean(abs(m.predict(X_test)-y_test.values.ravel())))

        #average internal model results and report as outer_result which*
        outer_results.append(np.mean(np.abs(model_scores)))

        #print('I Accuracy: %.3f (%.3f)' % (mean(inner_results), std(inner_results)))

    #*will in turn be averaged over all folds to determine that combination of term's outer score (real world performance, which will be our model evaluating metric).
    score = [np.mean(outer_results), np.std(outer_results)]
    #scores.append(([i,score]))
    return([i,score])
    
#for i in range(-92,92,1):


In [ ]:
npa0 = []

for t in range(-92,2,1):
    npa0.append([t,newDataDelta[['Adj Close']],newData[['VIXCLS']],0])
#print(npa)

compare_x_lag_y(npa0[0])

client0 = Client('192.168.3.100:8786')
#client.restart()

future0 = client0.map(compare_x_lag_y, npa0)

results0 = client0.gather(future0)

plt.plot(pd.DataFrame([r[1][0] for r in results0]).set_index(np.array(range(-92,2,1))))
plt.show()


In [ ]:
#results0

In [ ]:
np.array(range(-92,2,1))<0

In [ ]:
len(range(-92,2,1))

In [ ]:
range(-92,2,1)[np.array(range(-92,2,1))[np.array(range(-92,2,1))<0]]

In [ ]:
lag = range(-92,0,1)[np.argmin([r[1][0] for r in results0])]
print(lag)

In [ ]:
npa1 = []

#differenced y (twice)
for t in range(-92,0,1):
    npa1.append([t,newDataDelta[['Adj Close']],newData[['VIXCLS']],1])
#print(npa)

client1 = Client('192.168.3.100:8786')
#client.restart()

future1 = client1.map(compare_x_lag_y, npa1)

results1 = client1.gather(future1)

plt.plot(pd.DataFrame([r[1][0] for r in results1]).set_index(np.array(range(-92,0,1))))
plt.show()


In [ ]:
npa1 = []

#differenced y (twice)
for t in range(-92,0,1):
    npa1.append([t,newDataDelta[['Adj Close']],newDataDelta[['VIXCLS']],0])
#print(npa)

client1 = Client('192.168.3.100:8786')
#client.restart()

future1 = client1.map(compare_x_lag_y, npa1)

results1 = client1.gather(future1)

plt.plot(pd.DataFrame([r[1][0] for r in results1]).set_index(np.array(range(-92,0,1))))
plt.show()


In [ ]:
lag = range(-92,0,1)[np.argmin([r[1][0] for r in results1])]
print(lag)

In [ ]:
"""
plt.plot([abs(c.values[0][1]) for c in corrscores])
#plt.xticks(range(-92,92,1),[c.values[0][1] for c in corrscores], rotation=45)
plt.show()
range(-92,92,1)[np.argmax([abs(c.values[0][1]) for c in corrscores])]
"""

In [ ]:
#pd.concat([vix.reindex(early.index),sp500.reindex(early.index)['Adj Close'].pct_change().shift(range(-92,92,1)[np.argmin([s[1][0] for s in scores])])],axis=1).interpolate(method='time').dropna().corr()

In [ ]:
zca = ZCA()

In [ ]:

combined = pd.concat([newData[['VIXCLS']],newDataDelta[['Adj Close']].shift(lag)],axis=1).dropna()
print(combined.corr())
model = LinearRegression(fit_intercept=True)
#just getting hyper-parm's (fit)
lm = model.fit(zca.fit_transform(combined[['VIXCLS']]),combined[['Adj Close']])
print(lm.intercept_)
print(lm.coef_)

In [ ]:

combined = pd.concat([newDataDelta[['VIXCLS']],newDataDelta[['Adj Close']].shift(lag)],axis=1).dropna()
print(combined.corr())
model = LinearRegression(fit_intercept=True)
#just getting hyper-parm's (fit)
lm = model.fit(zca.fit_transform(combined[['VIXCLS']]),combined[['Adj Close']])
print(lm.intercept_)
print(lm.coef_)

In [ ]:
#dir(lm)

In [ ]:
plt.scatter(newDataDelta[['VIXCLS']].reindex(newDataDelta.index),newDataDelta[['Adj Close']])

In [ ]:
plt.scatter(newData[['VIXCLS']].reindex(newDataDelta.index),newDataDelta[['Adj Close']])

In [ ]:
plt.scatter(newDataDelta[['VIXCLS']].shift(lag).reindex(newDataDelta.index),newDataDelta[['Adj Close']])

In [ ]:
plt.scatter(newData[['VIXCLS']].shift(lag).reindex(newDataDelta.index),newDataDelta[['Adj Close']])

In [ ]:

#X = sm.add_constant(pd.concat([newDataDelta[['VIXCLS']],newDataDelta[['Adj Close']].shift(lag)],axis=1).dropna())[['VIXCLS']]
X = combined[['VIXCLS']]
y = combined[['Adj Close']]
results = sm.OLS(y,sm.add_constant(zca.fit_transform(X))).fit()
results.summary()  



In [ ]:
plt.scatter(results.predict(sm.add_constant(zca.fit_transform(X))),y)